In [1]:
#Imports
import pandas as pd
import numpy as np
import seaborn as sns

## GROUPBY

Some of the most important uses of a Pandas DataFrame involve grouping related data together and operating on the grouped subsets independently.
For example, data may be grouped by a categorical variable, such as state or county, and sales totals accumulated by the grouped region. 
To demonstrate this functionality, we turn to a second data set on restaurant data that is provided along with the seaborn Python module. 
The seaborn is presented in a separate lesson and provides support for advanced visualizations. 
Right now, however, we simply want to easily process this data, so we load the data into the dft DataFrame and 
display several randomly selected rows.

In [2]:
#Load data into DataFrame
dft = sns.load_dataset("tips")
dft.sample(4)

total_bill   tip   sex smoker   day    time  size
207       38.73  3.00  Male    Yes   Sat  Dinner     4
8         15.04  1.96  Male     No   Sun  Dinner     2
130       19.08  1.50  Male     No  Thur   Lunch     2
20        17.92  4.08  Male     No   Sat  Dinner     2

- To aggregate rows together, we employ the __groupby__ method to __create groups of rows that should be aggregated into a subset__. 
- The column (or columns) used to separate the rows are specified as a parameter to the groupby functions, as shown in the following code cell where the tips data set is grouped on the time column into a __new DataFrameGroupBy object__ called dg. 
- This new dg object can be operated on as a normal DataFrame with the exception that it contains subsets that are treated independently. 
- This is shown in the next two code blocks where the head and tail functions are used to show the first and last few rows of the group data set. 
Notice how the same number of rows are shown for each grouped data set.


In [42]:
# Group the DataFrame by the time column
dg = dft.groupby('time')
type(dg)

pandas.core.groupby.groupby.DataFrameGroupBy

In [44]:
# Display first three rows from each group
dg.head(3) 

total_bill   tip     sex smoker   day    time  size
0        16.99  1.01  Female     No   Sun  Dinner     2
1        10.34  1.66    Male     No   Sun  Dinner     3
2        21.01  3.50    Male     No   Sun  Dinner     3
77       27.20  4.00    Male     No  Thur   Lunch     4
78       22.76  3.00    Male     No  Thur   Lunch     2
79       17.29  2.71    Male     No  Thur   Lunch     2

### Operating on Groups

- The DataFrame groups can be operated by using similar techniques to the normal DataFrame.
For example, statistical quantities such as the median or standard deviation can be computed for each group, as shown in the next few code cells.

- Multiple functions can be computed at once by using the __aggregate method__, which __takes a list of the statistical functions to apply to each group__. 
- Finally, the __describe function__ can be applied to the groups, which __generates a statistical summary for each group of data__.


In [45]:
# Compute median for each column in each group
dg.median()

total_bill   tip  size
time                          
Lunch       15.965  2.25     2
Dinner      18.390  3.00     2

In [46]:
#Compute standard deviation for each column in each group
dg.std()

total_bill       tip      size
time                                  
Lunch     7.713882  1.205345  1.040024
Dinner    9.142029  1.436243  0.910241

In [47]:
#compute mean and std deviation for each column in each group
dg.aggregate(['mean','std'])

total_bill                 tip                size          
             mean       std      mean       std      mean       std
time                                                               
Lunch   17.168676  7.713882  2.728088  1.205345  2.411765  1.040024
Dinner  20.797159  9.142029  3.102670  1.436243  2.630682  0.910241

In [48]:
# Compute and display all summary statistics for all groups
dg.describe()

size                                                 tip            \
        count      mean       std  min  25%  50%  75%  max  count      mean   
time                                                                          
Lunch    68.0  2.411765  1.040024  1.0  2.0  2.0  2.0  6.0   68.0  2.728088   
Dinner  176.0  2.630682  0.910241  1.0  2.0  2.0  3.0  6.0  176.0  3.102670   

        ...                 total_bill                                      \
        ...       75%   max      count       mean       std   min      25%   
time    ...                                                                  
Lunch   ...    3.2875   6.7       68.0  17.168676  7.713882  7.51  12.2350   
Dinner  ...    3.6875  10.0      176.0  20.797159  9.142029  3.07  14.4375   

                                
           50%      75%    max  
time                            
Lunch   15.965  19.5325  43.11  
Dinner  18.390  25.2825  50.81  

[2 rows x 24 columns]

- Alternatively, one can determine the index of the rows that contain the minimum or maximum value on a group basis. For example:
- the two code cells below display the row index for the maximum (via the idxmax function) and the minimum (via the idxmin function) for each group.
- These functions return a DataFrame, thus the results can be extracted for an individual column in a group by using the standard DataFrame indexing techniques 


In [49]:
#compute row indices for column maximum values 
dg.idxmax()

total_bill    tip  size
time                           
Lunch        197.0  141.0   125
Dinner       170.0  170.0   156

In [50]:
# Compute row indices for column minimum values
dg.idxmin()

total_bill    tip  size
time                           
Lunch        149.0  135.0    82
Dinner        67.0   67.0    67

In [61]:
dft.total_bill.idxmax()

170

###################################

Q. First group the dft DataFrame by the sex column. Next, compute and display the minimum, maximum, and median values for the new grouped DataFrame by using the aggregate function correctly. Finally, compute the row index for the maximum total_bill for the Female group.

In [65]:
dsx = dft.groupby('sex')
dsx.aggregate(['min','max','median'])

total_bill                tip              size           
              min    max median  min   max median  min max median
sex                                                              
Male         7.25  50.81  18.35  1.0  10.0   3.00    1   6      2
Female       3.07  44.30  16.40  1.0   6.5   2.75    1   6      2

In [66]:
dsx.idxmax()

total_bill    tip  size
sex                            
Male         170.0  170.0   141
Female       102.0  214.0   125

In [67]:
dft.loc[102]

total_bill      44.3
tip              2.5
sex           Female
smoker           Yes
day              Sat
time          Dinner
size               3
Name: 102, dtype: object

## STACKING

- Given two or more DataFrame objects, a common task is joining them together. When working with a relational database, this is formally a __join__ operation, and Pandas supports joins across two DataFrame objects. 
- But __for two DataFrame objects that have the same structure__, the process can be simplified by employing either __horizontal stacking (where columns are combined)__ or __vertical stacking (where rows are combined)__. 
- These operations both use the Pandas __concat__ function, which by default assumes __axis=0, which implies vertical stacking.__ Specifying __axis=1 implies horizontal stacking, where columns from each subsequent DataFrame are added to the previous columns__.
- Note that this operation generates a new DataFrame with the concatenated data.

In [7]:
#chop tips dataset into two sets based on rows
tr1 = dft.loc[:200]
tr1.tail(2)

total_bill  tip   sex smoker   day   time  size
199       13.51  2.0  Male    Yes  Thur  Lunch     2
200       18.71  4.0  Male    Yes  Thur  Lunch     3

In [8]:
tr2 = dft.iloc[200:]
tr2.tail(2)

total_bill   tip     sex smoker   day    time  size
242       17.82  1.75    Male     No   Sat  Dinner     2
243       18.78  3.00  Female     No  Thur  Dinner     2

In [9]:
#vertical stacking
tv = pd.concat([tr1,tr2])

In [10]:
tv.head(3)

total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3

In [11]:
tv.describe()

total_bill         tip        size
count  245.000000  245.000000  245.000000
mean    19.781551    3.002367    2.571429
std      8.884417    1.382282    0.949547
min      3.070000    1.000000    1.000000
25%     13.370000    2.000000    2.000000
50%     17.810000    2.920000    2.000000
75%     24.080000    3.600000    3.000000
max     50.810000   10.000000    6.000000

In [12]:
tr1.describe()

total_bill         tip        size
count  201.000000  201.000000  201.000000
mean    19.800249    3.039701    2.572139
std      8.887140    1.347713    0.967481
min      3.070000    1.000000    1.000000
25%     13.810000    2.000000    2.000000
50%     17.890000    3.000000    2.000000
75%     23.950000    3.750000    3.000000
max     50.810000   10.000000    6.000000

In [13]:
tr2.describe()

total_bill        tip       size
count   44.000000  44.000000  44.000000
mean    19.696136   2.831818   2.568182
std      8.974092   1.535516   0.873318
min      7.740000   1.000000   1.000000
25%     12.865000   2.000000   2.000000
50%     16.435000   2.500000   2.000000
75%     26.065000   3.125000   3.000000
max     48.330000   9.000000   5.000000

In [14]:
# Split tips dataset based on columns

tc1 = dft.iloc[:,:2]
tc2 = dft.iloc[:,2:]

In [15]:
tc1.head(2)

total_bill   tip
0       16.99  1.01
1       10.34  1.66

In [16]:
tc2.head(2)

sex smoker  day    time  size
0  Female     No  Sun  Dinner     2
1    Male     No  Sun  Dinner     3

In [17]:
th = pd.concat([tc1,tc2],axis=1)

In [18]:
th.head(2)

total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3

## PIVOT TABLES

- A pivot table is a powerful technique for summarizing a large data set by grouping data together over multiple columns. 
- A pivot table can also display summary information for the new representation such as average or standard deviation. - Pivot tables are popular techniques when using spreadsheets, as they provide a quick and updatable summary of a potentially very large table of data.

- In Pandas, a pivot table can quickly be created by using the __pivot_table__ function. 
- This function takes one argument that specifies the column that should be aggregated. 
- The index argument is used to specify the row indices, which specifies how the DataFrame rows are grouped. 
- The columns argument specifies the columns that are displayed in the pivot table, and the aggfunc argument specifies which functions should be computed and displayed for the group data; default value is simply the average.

In [3]:
# Create Pivot Table with sex column as vertical index, day column as horizontal index
dft.pivot_table('total_bill', index='sex',columns='day')

day          Thur        Fri        Sat        Sun
sex                                               
Male    18.714667  19.857000  20.802542  21.887241
Female  16.715312  14.145556  19.680357  19.872222

In [4]:
dft.pivot_table('tip',index='sex',columns='time')

time       Lunch    Dinner
sex                       
Male    2.882121  3.144839
Female  2.582857  3.002115

In [5]:
dft.pivot_table('tip',index='sex',columns='smoker')

smoker       Yes        No
sex                       
Male    3.051167  3.113402
Female  2.931515  2.773519

In [6]:
dft.pivot_table('tip',index='day',columns='sex')

sex       Male    Female
day                     
Thur  2.980333  2.575625
Fri   2.693000  2.781111
Sat   3.083898  2.801786
Sun   3.220345  3.367222

In [8]:
# Create a Pivot Table using the sex column as the vertical index 
# and the day column as the horizontal index. This time include margin
# data for each row and column.

dft.pivot_table('total_bill',index='day',columns='sex',margins=all)

sex        Male     Female        All
day                                  
Thur  18.714667  16.715312  17.682742
Fri   19.857000  14.145556  17.151579
Sat   20.802542  19.680357  20.441379
Sun   21.887241  19.872222  21.410000
All   20.744076  18.056897  19.785943

In [9]:
# Create a Pivot Table using the day column as the horizontal index 
# and the sex and smoker columns as the hierarchical vertical index

dft.pivot_table('total_bill',index='day', columns=['sex','smoker'])

sex          Male                Female           
smoker        Yes         No        Yes         No
day                                               
Thur    19.171000  18.486500  19.218571  16.014400
Fri     20.452500  17.475000  12.654286  19.365000
Sat     21.837778  19.929063  20.266667  19.003846
Sun     26.141333  20.403256  16.540000  20.824286

In [10]:
# Create a Pivot Table using the sex column as the vertical index 
# and the day and smoker columns as the hierarchical horizontal index

dft.pivot_table('total_bill', index=['day', 'smoker'], columns='sex')

sex               Male     Female
day  smoker                      
Thur Yes     19.171000  19.218571
     No      18.486500  16.014400
Fri  Yes     20.452500  12.654286
     No      17.475000  19.365000
Sat  Yes     21.837778  20.266667
     No      19.929063  19.003846
Sun  Yes     26.141333  16.540000
     No      20.403256  20.824286

In [11]:
# Create a Pivot Table using the sex column as the horizontal index 
# and the day column as the horizontal index. Display aggregate max, 
# median, and min for the total_bill column.

dft.pivot_table(index='day', columns='sex', aggfunc={'total_bill': ['max', 'median', 'min']})

total_bill                                    
            max         median           min       
sex        Male Female    Male  Female  Male Female
day                                                
Thur      41.19  43.11  16.975  13.785  7.51   8.35
Fri       40.17  22.75  17.215  15.380  8.58   5.75
Sat       50.81  44.30  18.240  18.360  7.74   3.07
Sun       48.17  35.26  20.725  17.410  7.25   9.60